## Haplotype validation

Haplotype validation based on parent/progeny and synthetic X as a gold standard.

Recent updates:
- Read in raw phased data and draw bar plots from those.

In [1]:
%run setup.ipynb 

In [2]:
rcParams['text.usetex'] = False

In [3]:
from pathlib import Path
from matplotlib.ticker import FuncFormatter

In [4]:
def fposition(x, pos):
    'The two args are the value and tick position'
    return '%2d' % (x*1e-6)

In [5]:
autosomes = "2R", "2L", "3R", "3L"

In [6]:
meta_fn = Path(phase2_ar1.samples_dir) / "cross.samples.meta.txt"
meta_info = pandas.read_csv(meta_fn, sep="\t", index_col=0).set_index("ox_code")
meta_info.head()

,cross,role,n_reads,median_cov,mean_cov,sex,colony_id
ox_code,,,,,,,
AD0142-C,18-5,parent,60486753,26,25.824447,F,Ghana
AD0143-C,18-5,parent,58509103,19,18.800118,M,Kisumu/G3
AD0146-C,18-5,progeny,101612499,44,43.494594,NaN,NaN
AD0147-C,18-5,progeny,50710020,16,16.284487,NaN,NaN
AD0148-C,18-5,progeny,59023991,19,18.978021,NaN,NaN


In [7]:
meta_info.groupby(["cross", "role"]).size()

cross  role   
18-5   parent      2
       progeny    20
29-2   parent      2
       progeny    20
36-9   parent      2
       progeny    20
37-3   parent      2
       progeny    20
42-4   parent      2
       progeny    14
45-1   parent      2
       progeny    20
46-9   parent      2
       progeny    20
47-6   parent      2
       progeny    20
73-2   parent      2
       progeny    19
78-2   parent      2
       progeny    19
80-2   parent      2
       progeny    20
dtype: int64

In [8]:
# mem = memory.Memory(cachedir="/tmp")

In [9]:
# ihnfstem = os.path.join("/data/coluzzi/ag1000g/data/phase1/release/AR3.1",
#                         "haplotypes/crosses/hdf5",
#                         "{chrom}_{cross}_{parent}_inheritance")

In [10]:
callset_dir = Path(phase2_ar1.haplotypes_dir) / "crosses/zarr2"
callset_path = callset_dir / "ag1000g.crosses.phase2.ar1.haplotypes.zip"

In [11]:
callset_dir = Path("/kwiat/vector/ag1000g/analysis/20190410-phase2-phasing-par-pro/output")

In [12]:
# stor = zarr.ZipStore(callset_path, mode="r")
# callset = zarr.Group(stor)

In [13]:
xgp = meta_info.groupby("cross")

In [14]:
def plot_transmission(painting, ax=None, title=None, palette_name='colorblind'):

    palette = sns.color_palette(palette_name)

    # set figure height depending on number of haplotypes

    # map painting codes onto colours
    cmap = mpl.colors.ListedColormap([
        'white',      # 0 = undetermined
        palette[0],  # 1 = allele inherited from first parental haplotype
        palette[2],  # 2 = allele inherited from second parental haplotype
        'lightgrey',     # 3 = reference allele, also carried by both parental haplotypes
        'lightgrey',     # 4 = non-reference allele, also carried by both parental haplotypes
        'black',     # 5 = non-parental allele (i.e., Mendelian error)
        palette[5],  # 6 = either or both parental alleles missing
        'white',     # 7 = missing allele
    ])

    # plot painting
    ax.pcolormesh(painting.T, cmap=cmap, vmin=0, vmax=7)

    # tidy up axes
    ax.set_yticks(np.arange(painting.shape[1]) + .5)
    ax.set_yticklabels([])
    ax.set_xticklabels([])

    if title:
        ax.set_title(title)

In [15]:
genome = phase2_ar1.genome_agamp3

In [16]:
gs = GridSpec(
        3, len(autosomes), 
        height_ratios=(1, 1, 0.3),
        width_ratios=[len(genome[k]) for k in autosomes], 
        wspace=0.1, hspace=0.1)

In [17]:
def get_phased_genotypes(mother, father, progeny, fhcallset):
    
    samples = list(fhcallset["samples"].astype("<U8"))
    
    assert len(mother) == len(father) == 1, \
        "one mother and father must be specified"
    
    # first get progeny:
    proix = sorted([samples.index(s) for s in progeny])
    
    matix = [samples.index(s) for s in mother]
    patix = [samples.index(s) for s in father]
    
    gn = allel.GenotypeChunkedArray(fhcallset["calldata/genotype"])
    g_prime = allel.GenotypeChunkedArray(
        zarr.zeros((gn.shape[0], 2, 2), dtype="int8"))
    
    progeny_g = gn.take(proix, axis=1)
    g_prime[:, 0] = np.squeeze(gn.take(matix, axis=1))
    g_prime[:, 1] = np.squeeze(gn.take(patix, axis=1))
    g_prime.append(progeny_g, axis=1)
    
    return g_prime, allel.SortedIndex(fhcallset["variants/POS"])

In [18]:
def get_phased_genotypes(chrom, xid):
    
    callset_path = callset_dir / chrom / (xid + ".zarr2")
    
    callset = zarr.open_group(callset_path.as_posix(), mode="r")
    
    #callset = zarr.Group(stor)
    
    g = allel.GenotypeChunkedArray(callset["phased_genotypes"])
    p = allel.SortedIndex(callset["POS"])
    
    return g, p    

In [19]:
a, b = get_phased_genotypes("2L", "18-5")

In [20]:
met = meta_info.groupby(["cross"])

#met = meta_info.query("cross == '18-5'").groupby(["cross"])

In [21]:
from matplotlib.pyplot import GridSpec

In [22]:
gs = GridSpec(
    4, len(autosomes), 
    height_ratios=[5, 1, 5, 1],
    width_ratios=[len(genome[k]) for k in autosomes],
    wspace=0.1, hspace=0.1)

In [23]:
?allel.plot_variant_locator

In [24]:
for xid, ff in met:
    
    fig = plt.figure(figsize=(8, 4))

    plotfn =  Path("../artwork", "haplotype-validation",
                   "allele_transmission_{xid}.png".format(xid=xid))
    
    if plotfn.is_file():
        pass

    for i, chrom in enumerate(autosomes):
                
        phased_genotypes, pos = get_phased_genotypes(chrom, xid)
    
        het_loci = allel.GenotypeArray(phased_genotypes).is_het().any(axis=1)
        
        phased_genotypes = phased_genotypes.compress(het_loci, axis=0)

        haplotypes_progeny = allel.GenotypeArray(phased_genotypes[:, 2:, :])
        pos = pos.compress(het_loci)

        for ix, parent_label in enumerate(["Maternal", "Paternal"]):
            ax = fig.add_subplot(gs[ix * 2, i])
            

            # pull out mother's genotypes from the first column
            genotypes_parent = phased_genotypes[:, ix]

            # convert to haplotype array
            haplotypes_parent = genotypes_parent.to_haplotypes()

            # pull out maternal haplotypes from the progeny
            haplotypes_progeny_given_parent = haplotypes_progeny[:, :, ix]

            painting = allel.paint_transmission(
                haplotypes_parent, haplotypes_progeny_given_parent)

            #np.savez_compressed(cache_fn, painting=painting)
  
            # het_only
            hh = genotypes_parent.is_het().squeeze()
        
            painting = np.compress(hh, painting, axis=0)
            plot_pos = pos.compress(hh, axis=0)
            
            # skip every 2 rows.
            plot_transmission(painting, ax=ax)
            ax.tick_params(direction='out', length=1, width=1)

            if i == 0:
                ax.set_ylabel(parent_label)

            # make the locator plot...
            ax2 = fig.add_subplot(gs[ix * 2 + 1, i])
            allel.plot_variant_locator(plot_pos, ax=ax2, line_kwargs={"lw": 0.5})
            _ = ax2.set_xticks(np.arange(0, len(genome[chrom]), 1e7))
            ax2.xaxis.set_ticks_position("bottom")
            formatter = FuncFormatter(fposition)
            ax2.xaxis.set_major_formatter(formatter)
            ax2.set_xlabel(chrom)

    _ = fig.text(0.5, 0.03, 'Genomic position (Mbp)', ha='center', va='top')
    _ = fig.suptitle("Haplotype transmission for {xid}".format(xid=xid))

    fig.savefig(
        plotfn,
        dpi=250,
        bbox_inches='tight')
    
    plt.close("all")

## Panel B

In [ ]:
from intervaltree import IntervalTree
import pyfasta

In [ ]:
import re
def find_reference_gaps(genome_fa, gap_size=300):

    # fix on a gapsize of 1kb, as ld breaks down over this distance and
    # do chunks of 10k at a time. look for consecutive Ns.
    size = 10000
    gaps = list()
    
    match = "N{{{n}}}?".format(n=gap_size)
    
    for i in range(0, len(genome_fa), size - (2 * gap_size)):
        text = genome_fa[i:i+size]

        for m in re.finditer(match, text, flags=re.IGNORECASE):
            gaps.append((i + m.start(), i + m.end()))

    tree = IntervalTree.from_tuples(gaps)
    tree.merge_overlaps()

    return np.array([[iv.begin, iv.end] for iv in sorted(tree)])

In [ ]:
# load data from data release

In [ ]:
eval_path = Path(
    "/kwiat/vector/ag1000g/analysis/20170316-phase2-haplotypes")

In [ ]:
frame_d = {
    chrom: eval_path / "eval_autosomal_phasing/output/{chrom}_table_switch_errors.txt".format(chrom=chrom)
    for chrom in autosomes}

frame_d["X"] = eval_path / "eval_x_phasing/output/X_table_switch_errors.txt"

In [ ]:
for k, v in frame_d.items():
    print(k, v.absolute(), v.is_file())   

In [ ]:
def draw_msd_across_genome(genome_pos, marker_dist, mean_switch_dist, number_markers, chrom, ax):

    mean_switch_dist[number_markers < 50] = np.NaN
        
    ax.plot(genome_pos, 2 * marker_dist, 'k-', linewidth=1.0)
    ax.plot(genome_pos, mean_switch_dist, 'r-',  linewidth=1.0)

    ax.grid(True)
    
    ax.set_yscale("log")
    ax.set_ylim((100, 2000000))
    ax.set_xlim((0, len(genome[chrom])))
    
    _ = ax.set_xticks(np.arange(0, len(genome[chrom]), 1e7))

    return ax

In [ ]:
def draw_switch_err_rate(genome_pos, rate, number_markers, chrom, ax):

    rate[number_markers < 50] = np.NaN   
    ax.plot(genome_pos, 1 - rate, 'r-', linewidth=1.0)

    ax.grid(True)
    ax.set_ylim((0.82, 1.0))
    ax.set_xlim((0, len(genome[chrom])))
    
    _ = ax.set_xticks(np.arange(0, len(genome[chrom]), 1e7))
    formatter = FuncFormatter(fposition)
    ax.xaxis.set_major_formatter(formatter)

    return ax

In [ ]:
contigs = chromosomes[:5]


gs = GridSpec(
    2, len(contigs), 
    width_ratios=[len(genome[k]) for k in contigs], 
    wspace=0.2)

fig = plt.figure(figsize=(12, 3))

for i, chrom in enumerate(contigs):

    df = pandas.read_csv(frame_d[chrom], sep="\t")

    ax1 = fig.add_subplot(gs[0, i])
    ax2 = fig.add_subplot(gs[1, i])
    sns.despine(ax=ax1, offset=5)
    sns.despine(ax=ax2, offset=5)

    pos = 0.5 * np.array(df.start + df.stop)
    mean_marker_d = np.array(df.mean_marker_dist)
    mean_switch_d = np.array(df.mean_switch_dist)

    n_markers = np.array(df.n_markers)
    err_rate = np.array(df.err_rate)

    draw_msd_across_genome(pos, mean_marker_d, mean_switch_d, n_markers, chrom, ax1)
    draw_switch_err_rate(pos, err_rate, n_markers, chrom, ax2)

    if i == 0:
        ax1.set_ylabel("Mean switch\ndistance (bp)")
        ax2.set_ylabel("1 - \n(switch error rate)")
    else:
        ax1.yaxis.set_tick_params(which="both", width=0)
        ax2.yaxis.set_tick_params(which="both", width=0)

        ax1.set_yticklabels([])
        ax2.set_yticklabels([])

    ax1.xaxis.set_tick_params(which="both", width=0)
    ax1.set_xticklabels([])
    ax1.set_title(chrom, fontsize=10)

fig.text(0.5, -0.04, 'Genomic position (Mbp)', ha='center', va='center', fontsize=10)

fig.savefig("../artwork/haplotype-validation/switch_across_genome.png", 
            dpi=300, bbox_inches='tight')

## Finally compute the per chromosome means presented in the paper

In [ ]:
qdf = pandas.concat({chrom: pandas.read_csv(
            frame_d[chrom], sep="\t") for chrom in autosomes})

qdf

In [ ]:
summed = qdf.sum()

In [ ]:
# this is the autosomal mean switch distance
mean_switch_dist = (summed.distance/summed.n_markers)/(summed.n_errors/summed.n_markers)
mean_switch_dist

In [ ]:
rrate = 0.983

In [ ]:
(mean_switch_dist/1e6) * rrate

In [ ]:
def haldane(r):
    return -np.log(1- (2 * r))/2

In [ ]:
summed.n_errors/summed.n_markers

In [ ]:
haldane(summed.n_errors/summed.n_markers)

In [ ]:
xdf = pandas.read_csv(frame_d["X"], sep="\t")

In [ ]:
xsum = xdf.sum(0)

In [ ]:
pd.DataFrame(xsum).T.eval("(distance/n_markers)/(n_errors/n_markers)")

In [ ]:
haldane(xsum.n_errors/xsum.n_markers)